In [62]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

### Features Detection

In [3]:
sg_tsv = './features_detection/training/train.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [ ]:
'''mylist = []
sg_tsv = './features_detection/training/train.feature.tsv'
for chunk in pd.read_csv(sg_tsv, sep='\t', header = None, converters={1:json.loads}, chunksize=20000):
    mylist.append(chunk)

df_feature_detection = pd.concat(mylist, axis= 0)
del mylist
df_feature_detection=df_feature_detection.rename(columns={0: "id", 1: "features"})
#df_feature_detection'''

In [4]:
sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [ ]:
'''sg_tsv = './features_detection/training/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection'''

In [5]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1000092795,{'features': 'FUTwPG3ezD8AAAAAAAAAAKR5DEAAAAAA...,"[{'class': 'Bush', 'conf': 0.8409668207168579,..."
1,10002456,{'features': 'AAAAAGPqez4AAAAACHhsPigtHz/J+MI/...,"[{'class': 'Sky', 'conf': 0.7663399577140808, ..."
2,1000268201,{'features': 'RcyMPgAAAAAAAAAAIrObQAAAAAAAAAAA...,"[{'class': 'Dress', 'conf': 0.9922232031822205..."
3,1000344755,{'features': 'R/6JPkusXj/A7KA/AAAAAAAAAAAAAAAA...,"[{'class': 'Man', 'conf': 0.9483818411827087, ..."
4,1000366164,{'features': 'AAAAAAAAAAAAAAAAAAAAABVWmz0AAAAA...,"[{'class': 'Glasses', 'conf': 0.96930736303329..."
...,...,...,...
995,1342780478,{'features': 'AAAAAAAAAAAAAAAArKxqPqx+Ez4AAAAA...,"[{'class': 'Tie', 'conf': 0.947785496711731, '..."
996,134283905,{'features': 'AAAAAAAAAAAAAAAAAAAAAASW1kAAAAAA...,"[{'class': 'Cloud', 'conf': 0.9414686560630798..."
997,1343787097,{'features': 'FwW7PAAAAAAAAAAAAAAAAAAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.8657943606376648,..."
998,1344692842,{'features': 'cMn8PgAAAAD3Wp8/d7ECPagTpj8AAAAA...,"[{'class': 'Stick', 'conf': 0.894828736782074,..."


In [ ]:
#df_split = np.array_split(full_detection, 3)

In [ ]:
#full_detection_1 = df_split[2]

In [6]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:03, 321.32it/s]


### Features Segmentation

In [7]:
sg_tsv = './features_test1/training/train.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [ ]:
'''mylist = []
sg_tsv = './features_test1/training/train.feature.tsv'
for chunk in pd.read_csv(sg_tsv, sep='\t', header = None, converters={1:json.loads}, chunksize=20000):
    mylist.append(chunk)

df_feature_segmentation = pd.concat(mylist, axis= 0)
del mylist
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})
#df_feature_segmentation'''

In [8]:
sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, nrows=1000)
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [ ]:
'''sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation'''

In [9]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...,...
995,1342780478,{'features': 'eBWZQPe+EUEJgwBBmmijQRgZYEBifZlA...,"[{'class': 'Necktie', 'conf': 0.97876840829849..."
996,134283905,{'features': 'VkC+QKD3dUDsH9o+aFtEQf5N7j8AAAAA...,"[{'class': 'Jacket', 'conf': 0.509735286235809..."
997,1343787097,{'features': 'XgvoQL1xeUAKm0ZADnFqPwAAAAAAAAAA...,"[{'class': 'Flag', 'conf': 0.9085625410079956,..."
998,1344692842,{'features': 'FiohQCRezz+2FMM/+/AkQW4OdkDqUps/...,"[{'class': 'Suitcase', 'conf': 0.8603649139404..."


In [ ]:
#df_split_seg = np.array_split(full_segmentation, 3)

In [ ]:
#full_segmentation_1 = df_split_seg[2]

In [10]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1000it [00:01, 893.15it/s]


In [11]:
# analysis

In [12]:
len(full_detection.iloc[0]["label"])

37

In [18]:
for elem in full_segmentation.iloc[0]["label"]:
    similar_region = {"class": "", "score": 0}
    for elem2 in full_detection.iloc[0]["label"]:
        if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > 0.985:
            print("class segmentation: ", elem["class"])
            print("class detection: ", elem2["class"])
            similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
            print("similarity: ", similarity)
            if (similarity > similar_region["score"]):
                similar_region["score"] = similarity
                similar_region["class"] = elem2["class"]
            print("______________________________________________________________________")
    if similar_region["class"] != 0:
        print("======================================================================")
        print("Classe vecchia: ", elem["class"])
        print("Classe nuova: ", similar_region["class"])
        print("======================================================================")

class segmentation:  Belt buckle
class detection:  Shirt
similarity:  0.9921498332262234
______________________________________________________________________
class segmentation:  Belt buckle
class detection:  Shoe
similarity:  0.9893416487890953
______________________________________________________________________
class segmentation:  Belt buckle
class detection:  Jean
similarity:  0.9860327017523645
______________________________________________________________________
class segmentation:  Belt buckle
class detection:  Hand
similarity:  0.9969346398600796
______________________________________________________________________
class segmentation:  Belt buckle
class detection:  Hand
similarity:  0.9993740791335165
______________________________________________________________________
class segmentation:  Belt buckle
class detection:  Sneaker
similarity:  0.9890558682561952
______________________________________________________________________
class segmentation:  Belt buckle
class det

In [137]:
full_detection.iloc[index]["label"]

[{'class': 'Short',
  'conf': 0.966752827167511,
  'rect': [170.19406127929688,
   236.1067657470703,
   254.9027862548828,
   275.9931640625],
  'feature': array([0.    , 0.4147, 0.    , ..., 0.552 , 0.0798, 0.2529], dtype=float32),
  'bbox_id': 290},
 {'class': 'Shadow',
  'conf': 0.9499807953834534,
  'rect': [15.847036361694336,
   365.7681884765625,
   191.41802978515625,
   478.12506103515625],
  'feature': array([14.9565,  0.1622,  0.0506, ...,  0.9563,  0.2247,  0.5241],
        dtype=float32),
  'bbox_id': 291},
 {'class': 'Wheel',
  'conf': 0.8906331062316895,
  'rect': [181.47726440429688,
   458.8328552246094,
   198.1981658935547,
   478.4175109863281],
  'feature': array([0.0227, 1.6149, 0.    , ..., 0.9568, 0.0392, 0.0499], dtype=float32),
  'bbox_id': 292},
 {'class': 'Wheel',
  'conf': 0.8440861105918884,
  'rect': [246.96534729003906,
   434.4931945800781,
   264.33465576171875,
   451.96929931640625],
  'feature': array([0.    , 0.0885, 0.    , ..., 0.9039, 0.035 , 0

In [136]:
index = 7
nlp = en_core_web_md.load()
for elem in full_segmentation.iloc[index]["label"]:
    synonyms = []
    similar_region = {"class": "", "score": 0, "feature": [], "conf": 0}
    similar_regions = {}
    similar_regions_features = {}
    similar_features = {"class": "", "score": 0, "feature": [], "conf": 0}
    for syn in wordnet.synsets(elem["class"]):
        for l in syn.lemmas():
            synonyms.append(l.name())
    for elem2 in full_detection.iloc[index]["label"]:
        if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > 0.985:
            #print("class segmentation: ", elem["class"])
            #print("class detection: ", elem2["class"])
            similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
            #print("similarity: ", similarity)
            sim_features = 1 - spatial.distance.cosine(elem["feature"], elem2["feature"])
            #print("similarity features: ", 1 - spatial.distance.cosine(elem["feature"], elem2["feature"]))
            if (sim_features > similar_features["score"]): # 3
                similar_features["score"] = sim_features
                similar_features["class"] = elem2["class"]
                similar_features["feature"] = elem2["feature"]
                similar_features["conf"] = elem2["conf"]
            if (similarity > similar_region["score"]): # 1
                similar_region["score"] = similarity
                similar_region["class"] = elem2["class"]
                similar_region["feature"] = elem2["feature"]
                similar_region["conf"] = elem2["conf"]
            if elem2["class"] in similar_regions: # 2
                similar_regions[elem2["class"]] += 1
                if similarity > similar_regions_features[elem2["class"]]["score"]: 
                    similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
            else:
                similar_regions[elem2["class"]] = 0
                similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
            #print("______________________________________________________________________")
    if similar_region["class"] != "":
        print("\n======================================================================")
        #print("sinonimi :", synonyms)
        print("Classe vecchia: ", elem["class"])
        classe = max(similar_regions, key=similar_regions.get)
        print("Classe nuova metodo 1: ", classe)
        sim1 = 1 - spatial.distance.cosine(elem["feature"], similar_regions_features[classe]["feature"])
        #print("Similarity fetures 1: ", sim1)
        #print("Conf 1: ", similar_regions_features[classe]["conf"])
        tokens = nlp(elem["class"].lower() + " " + classe.lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_1 = token1.similarity(token2)* 1.2
        #print("Similarity words 2: ", similarity_seg_1)
        print("Classe nuova metodo 2: ", similar_region["class"])
        sim2 = 1 - spatial.distance.cosine(elem["feature"], similar_region["feature"])
        #print("Similarity fetures 2: ", sim2)
        #print("Conf 2: ", similar_region["conf"])
        tokens = nlp(elem["class"].lower() + " " + similar_region["class"].lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_2 = token1.similarity(token2)* 1.2
        #print("Similarity words 2: ", similarity_seg_2)
        print("Classe nuova metodo 3: ", similar_features["class"])
        #print("Similarity fetures 3: ", similar_features["score"])
        #print("Conf 3: ", similar_features["conf"])
        tokens = nlp(elem["class"].lower() + " " + similar_features["class"].lower())
        token1, token2 = tokens[0], tokens[1]
        similarity_seg_3 = token1.similarity(token2)* 1.2
        #print("Similarity words 3: ", similarity_seg_3)
        
        score_1_sim_feat = similarity_seg_1
        score_1_sim_word = sim1
        score_1_conf = similar_regions_features[classe]["conf"]/5
        score_1_tot = (score_1_sim_feat + score_1_sim_word + score_1_conf)
        
        score_2_sim_feat = similarity_seg_2
        score_2_sim_word = sim2
        score_2_conf = similar_region["conf"]/5
        score_2_tot = (score_2_sim_feat + score_2_sim_word + score_2_conf)
        
        score_3_sim_feat = similarity_seg_3
        score_3_sim_word = similar_features["score"]
        score_3_conf = similar_features["conf"]/5
        score_3_tot = (score_3_sim_feat + score_3_sim_word + score_3_conf)
        
        #print("111111111111111111111111111111111111111111111111111111111111")
        #print("score 1:\n", score_1_tot)
        #print("sim feat: ",score_1_sim_feat)
        #print("sim word: ",score_1_sim_word)
        #print("conf: ",score_1_conf)
        #print("22222222222222222222222222222222222222222222222222222222222")
        #print("score 2:\n", score_2_tot)
        #print("sim feat: ",score_2_sim_feat)
        #print("sim word: ",score_2_sim_word)
        #print("conf: ",score_2_conf)
        #print("333333333333333333333333333333333333333333333333333333333333")
        #print("score 3:\n", score_3_tot)
        #print("sim feat: ",score_3_sim_feat)
        #print("sim word: ",score_3_sim_word)
        #print("conf: ",score_3_conf)
        print("------------------------------------------------------------")
        if elem["class"] == classe:
            print("classe finale la 1 corrispondenza esatta: ", classe)
        elif elem["class"] == similar_region["class"]:
            print("classe finale la 2 corrispondenza esatta: ", similar_region["class"])
        elif elem["class"] == similar_features["class"]:
            print("classe finale la 3 corrispondenza esatta: ", similar_features["class"])
        elif classe.lower() in synonyms:
            print("classe finale la 1 corrispondenza sinonimi: ", classe)
        elif score_1_tot >= score_2_tot and score_1_tot >= score_3_tot:
            print("classe finale la 1 word similarity: ", classe)
        elif score_2_tot >= score_1_tot and score_2_tot >= score_3_tot:
            print("classe finale la 2 word similarity: ", similar_region["class"])
        elif score_2_tot >= score_1_tot and score_3_tot >= score_2_tot:
            print("classe finale la 3 word similarity: ", similar_features["class"])
        elif similar_region["class"].lower() in synonyms:
            print("classe finale la 2 corrispondenza sinonimi: ", similar_region["class"])
        elif similar_region["class"].lower() in synonyms:
            print("classe finale la 3 corrispondenza sinonimi: ", similar_features["class"])
        elif (sim1 >= sim2 and sim1 >= similar_features["score"]):
            print("classe finale la 1: ", classe)
        elif (similar_features["score"] >= sim2 and similar_features["score"] >= sim1):
            print("classe finale la 3: ", similar_features["class"])
        else: 
            print("classe finale la 2: ", similar_region["class"])
        print("======================================================================\n")
        
        img = str(full_detection.iloc[index]["id"])+".jpg"
        im_cv2 = cv2.imread("../datasets/flickr30k_images/train/"+img)
        x,y,w,h = tuple(elem["rect"])
        ROI = im_cv2[int(y):int(h), int(x):int(w)]
        #print(str(elem["class"]))
        #print("area: ", ROI.shape[0]*ROI.shape[1])
        cv2.imshow(str(elem["class"]),ROI)
        cv2.waitKey(0) 
        cv2.destroyAllWindows() 


Classe vecchia:  Cell phone
Classe nuova metodo 1:  Light
Classe nuova metodo 2:  Phone
Classe nuova metodo 3:  Head
------------------------------------------------------------
classe finale la 3 word similarity:  Head


Classe vecchia:  Necklace
Classe nuova metodo 1:  Light
Classe nuova metodo 2:  Necklace
Classe nuova metodo 3:  Neck
------------------------------------------------------------
classe finale la 2 corrispondenza esatta:  Necklace


Classe vecchia:  Short pants
Classe nuova metodo 1:  Short
Classe nuova metodo 2:  Short
Classe nuova metodo 3:  Leg
------------------------------------------------------------
classe finale la 1 word similarity:  Short


Classe vecchia:  Button
Classe nuova metodo 1:  Light
Classe nuova metodo 2:  Necklace
Classe nuova metodo 3:  Hand
------------------------------------------------------------
classe finale la 3 word similarity:  Hand


Classe vecchia:  Taillight
Classe nuova metodo 1:  Light
Classe nuova metodo 2:  Light
Classe nuova 

In [74]:
for syn in wordnet.synsets("man"):
    for l in syn.lemmas():
        synonyms.append(l.name())
print(synonyms)

['person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'person', 'individual', 'someone', 'somebody', 'mortal', 'soul', 'person', 'person', 'man', 'adult_male', 'serviceman', 'military_man', 'man', 'military_personnel', 'man', 'homo', 'man', 'human_being', 'human', 'man', 'man', 'valet', 'valet_de_chambre', 'gentleman', "gentleman's_gentleman", 'man', 'man', 'Man', 'Isle_of_Man', 'man', 'piece', 'world', 'human_race', 'humanity', 'humankind', 'human_beings', 'humans', 'mankind', 'man', 'man', 'man']


In [96]:
tokens = nlp("Jacket person")
  
token1, token2 = tokens[0], tokens[1]
  
print("Similarity:", token1.similarity(token2))

Similarity: 0.31442162


C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_10472/2458834535.py:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print("Similarity:", token1.similarity(token2))


In [116]:
import en_core_web_md
nlp = en_core_web_md.load()

doc1 = nlp("grass")
doc2 = nlp("ground")

print(doc1.similarity(doc2))

0.5477709058673111


In [93]:
words

'man person'

In [89]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
tokens = nlp(["man", "person"])
  
print("Similarity:", tokens[0].similarity(tokens[1]))

ValueError: [E866] Expected a string or 'Doc' as input, but got: <class 'list'>.

### Union

In [ ]:
full_not_present = []
for i in tqdm(range(len(full_detection))):    
    not_present = []
    for row_dect in full_detection.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985:
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

In [43]:
'''for elem in full_detection_1["id"].values == full_segmentation_1["id"].values:
    if not elem: 
        print("noooo")'''

In [15]:
#len(full_segmentation["label"].iloc[0])

In [16]:
#len(full_detection.iloc[0]["label"])

In [17]:
#len(full_not_present[0])

In [18]:
#full_segmentation.iloc[0]["label"]

In [19]:
#full_detection.iloc[0]["label"]

In [20]:
#full_not_present[0]

In [ ]:
full_segmentation["not_present"] = full_not_present

In [22]:
#len(full_segmentation["not_present"].iloc[3][0]["feature"])

In [ ]:
full_segmentation["union"] = full_segmentation["label"] + full_segmentation["not_present"]
full_segmentation

In [46]:
for index, row in tqdm(full_segmentation_1.iterrows()):
    if(len(row["label"]) + len(row["not_present"]) != len(row["union"])):
        print("!!!!!!!!!!!!!!!")

9666it [00:01, 6458.34it/s]


In [47]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [48]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [49]:
full_segmentation_1['feature_union'] = full_segmentation_1.progress_apply(generate_features,axis=1)
full_segmentation_1['feature_union'] = full_segmentation_1['feature_union'].progress_apply(json.dumps)

#full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
#full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9666/9666 [01:24<00:00, 113.73it/s]


In [2]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'train.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'train3.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [51]:
tsv_writer(full_segmentation_1[["id", "feature_union"]].values.tolist(), FEATURE_FILE)

In [85]:
tsv_writer(full_segmentation[["id", "label_union"]].values.tolist(),LABEL_FILE)

In [ ]:
# union splits

In [4]:
sg_tsv = './training_union/train1.feature.tsv'
df_features_1 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_1

,0,1
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
9662,3030793171,{'features': '/gmGQKg0GD8SM91AtK7JQFulWEEIf0dA...
9663,3030823649,{'features': 'PCxUPwHc8EDw4WE/AAAAAE5tsUCCtdhA...
9664,3030824089,{'features': 'lbLMQB7/CkELuZRAAAAAAAKA+EBIPm1A...
9665,3030953639,{'features': 'q7wGQU84N0GA+W1AAAAAAGZeY0FoQCdA...


In [8]:
sg_tsv = './training_union/train2.feature.tsv'
df_features_2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_2

,0,1
0,3030954495,{'features': 'd6ljQPbrSj+VxRlAgil4QIYJhEGmQ0s/...
1,3030962048,{'features': 'fMXQP9tKC0FIkrhAk38GQdKMAj9eL7VA...
2,3031011217,{'features': 'AAAAADqGmEDgH8o/AAAAAFZUCUDvZLNA...
3,303102907,{'features': 'AAAAAAAAAAAAAAAALAnIPwAAAADhkHpA...
4,303103123,{'features': '7BaPQC45OkAiKV1AdOxgQODzzEAzKntA...
...,...,...
9662,4613097862,{'features': '0HYvPwRn0EAAAAAAyPcRQU1CNUC8LJE/...
9663,4613186973,{'features': 'LCo3QBMn7kDlAClBM6DJPxR/wT8AsCI6...
9664,4613262585,{'features': '2nn/QFup10CJTxxBoYWMP9o14kBNH5BA...
9665,4613263773,{'features': 'gKxiQVtMlT+lvuRATPJwQOBm00DOVftA...


In [9]:
sg_tsv = './training_union/train3.feature.tsv'
df_features_3 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_features_3

,0,1
0,4613264477,{'features': 'sHcLQObNPEAAAAAA4IaZPmVkuUAj1s9A...
1,4613268823,{'features': 'hOHAQD+vBEFX63RAXpr8QKIX6z+yFwg/...
2,4613883258,{'features': 'RvwoQZRc8T8gZkw+2vXeQF0Gkj94nFpA...
3,4613887642,{'features': '8KK8QDYZGkFo49JAAAAAABHIz0AG3Q1A...
4,4613916679,{'features': 'tnHoP+q4h0GY7lI/Nk9CPxrbAkD1wghA...
...,...,...
9661,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
9662,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
9663,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
9664,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [12]:
df_features = df_features_1.append([df_features_2, df_features_3], ignore_index=True).rename(columns={0: "id", 1: "features"})
df_features

,id,features
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [18]:
df_features['features'] = df_features['features'].apply(json.dumps)

In [20]:
OUTPUT_DIR = './training_union/'

FEATURE_FILE = os.path.join(OUTPUT_DIR,'train.feature.tsv')

tsv_writer(df_features[["id",'features']].values.tolist(),FEATURE_FILE)

In [13]:
sg_tsv = './training_union/train.label.tsv'
df_labels = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_labels = df_labels.rename(columns={0: "id", 1: "label"})
df_labels

,id,label
0,1000092795,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...
28995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [14]:
df_full = pd.merge(df_features, df_labels, on="id")
df_full

,id,features,label
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...,"[{'class': 'Belt buckle', 'conf': 0.5350186228..."
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...,"[{'class': 'Rubber band', 'conf': 0.7075430750..."
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...,"[{'class': 'Drum (musical instrument)', 'conf'..."
...,...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [16]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(df_full.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

29000it [05:39, 85.46it/s] 
